In [4]:
from pymilvus import MilvusClient                                                            
client = MilvusClient(
    uri='https://in03-17ede2a458223b0.serverless.aws-eu-central-1.cloud.zilliz.com:443',     
    token='db_17ede2a458223b0:Hd7<b<]BzrR1kJe]'
)
print(client.list_collections())

[]


In [5]:
from pymilvus import MilvusClient
client = MilvusClient(
    uri='https://in03-17ede2a458223b0.serverless.aws-eu-central-1.cloud.zilliz.com:443',
    token='00c57898f2d095022b9efe7ea683742d0881d226f201625910518d6c6ffc52ab20a5f5bf3ec69b1442209acef625895444484f15'
)
print(client.list_collections())

[]


In [6]:
from pymilvus import MilvusClient
import time

# Source: local Milvus
src = MilvusClient(uri="http://localhost:19530")

# Destination: Zilliz Cloud
dst = MilvusClient(
    uri="https://in03-17ede2a458223b0.serverless.aws-eu-central-1.cloud.zilliz.com:443",
    token="db_17ede2a458223b0:Hd7<b<]BzrR1kJe]"
)

# List all collections
collections = src.list_collections()
print("Collections found:", collections)

Collections found: ['lebanese_laws']


In [7]:
from pymilvus import MilvusClient, DataType
import time

# Source: local Milvus
src = MilvusClient(uri="http://localhost:19530")

# Destination: Zilliz Cloud
dst = MilvusClient(
    uri="https://in03-17ede2a458223b0.serverless.aws-eu-central-1.cloud.zilliz.com:443",
    token="db_17ede2a458223b0:Hd7<b<]BzrR1kJe]"
)

COLLECTION = "lebanese_laws"
BATCH_SIZE = 100

# Step 1: Get source collection schema
print("Getting schema...")
schema_info = src.describe_collection(COLLECTION)
print(schema_info)

Getting schema...
{'collection_name': 'lebanese_laws', 'auto_id': True, 'num_shards': 1, 'description': 'Lebanese Legal Assistant Knowledge Base', 'fields': [{'field_id': 100, 'name': 'pk', 'description': 'Unique identifier', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': 'Embedding vector', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}, {'field_id': 102, 'name': 'source_type', 'description': 'Source type (e.g., ruling, article)', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 64}, 'is_partition_key': True}, {'field_id': 103, 'name': 'text_content', 'description': 'The actual text content', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'field_id': 104, 'name': 'metadata', 'description': 'Additional metadata', 'type': <DataType.JSON: 23>, 'params': {}}], 'functions': [], 'aliases': [], 'collection_id': 464412669616609246, 'consistency_level': 2, 'propertie

In [8]:
from pymilvus import MilvusClient, DataType
import time

# Source: local Milvus
src = MilvusClient(uri="http://localhost:19530")

# Destination: Zilliz Cloud
dst = MilvusClient(
    uri="https://in03-17ede2a458223b0.serverless.aws-eu-central-1.cloud.zilliz.com:443",
    token="db_17ede2a458223b0:Hd7<b<]BzrR1kJe]"
)

COLLECTION = "lebanese_laws"
BATCH_SIZE = 100

# Step 1: Create collection on Zilliz with same schema
print("Creating collection on Zilliz...")

schema = MilvusClient.create_schema(auto_id=True, enable_dynamic_field=False)
schema.add_field("pk", DataType.INT64, description="Unique identifier", is_primary=True, auto_id=True)
schema.add_field("vector", DataType.FLOAT_VECTOR, dim=1536, description="Embedding vector")
schema.add_field("source_type", DataType.VARCHAR, max_length=64, description="Source type (e.g., ruling, article)", is_partition_key=True)
schema.add_field("text_content", DataType.VARCHAR, max_length=65535, description="The actual text content")
schema.add_field("metadata", DataType.JSON, description="Additional metadata")

index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="vector",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

# Drop if exists and recreate
existing = dst.list_collections()
if COLLECTION in existing:
    print(f"Collection already exists on Zilliz, dropping...")
    dst.drop_collection(COLLECTION)

dst.create_collection(
    collection_name=COLLECTION,
    schema=schema,
    index_params=index_params,
    consistency_level="Strong"
)
print("Collection created on Zilliz!")

# Step 2: Count source data
total = src.query(COLLECTION, filter="pk > 0", output_fields=["count(*)"])
print(f"Total records to migrate: {total}")

# Step 3: Migrate data in batches
print("Starting migration...")
offset = 0
total_inserted = 0

while True:
    rows = src.query(
        collection_name=COLLECTION,
        filter="pk > 0",
        output_fields=["pk", "vector", "source_type", "text_content", "metadata"],
        limit=BATCH_SIZE,
        offset=offset
    )
    
    if not rows:
        break
    
    # Remove pk so Zilliz auto-generates new ones
    for row in rows:
        row.pop("pk", None)
    
    dst.insert(collection_name=COLLECTION, data=rows)
    total_inserted += len(rows)
    offset += BATCH_SIZE
    print(f"Migrated {total_inserted} records...")
    time.sleep(0.1)  # avoid rate limiting

print(f"Migration complete! Total inserted: {total_inserted}")

Creating collection on Zilliz...
Collection created on Zilliz!
Total records to migrate: data: ["{'count(*)': 1273}"], extra_info: {}
Starting migration...
Migrated 100 records...
Migrated 200 records...
Migrated 300 records...
Migrated 400 records...
Migrated 500 records...
Migrated 600 records...
Migrated 700 records...
Migrated 800 records...
Migrated 900 records...
Migrated 1000 records...
Migrated 1100 records...
Migrated 1200 records...
Migrated 1273 records...
Migration complete! Total inserted: 1273
